In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени\
from bs4 import BeautifulSoup
import random
import re
import json

In [3]:
page_link = 'https://simplewine.ru/catalog/vino/filter/color-krasnoe/page1/'


In [3]:
!pip install fake_useragent
!pip install beautifulsoup4

  Obtaining dependency information for fake_useragent from https://files.pythonhosted.org/packages/33/c9/ff44922639b8827dbc86d463d870dabfc19d1567d8a6427dcb2289d83fd8/fake_useragent-1.4.0-py3-none-any.whl.metadata


In [4]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 OPR/101.0.0.0'

In [5]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().safari})
response

<Response [200]>

In [6]:
# Функция, которая парсит бесплатные прокси.
# На вход получает html
# Возвращает список с прокси. Протокол выбран только http
def get_proxies(html):
    final_proxies_list = []
    soup = BeautifulSoup(html.content, 'html.parser')
    proxies_list = soup.find('tbody').find_all('tr')
    for j in proxies_list:
        proxy_row = j.find_all('td')
        ip = proxy_row[0].text
        port = proxy_row[1].text
        if proxy_row[6].text == 'no':
            protocol = 'http'
            proxy = protocol + '://' + ip + ':' + port
            final_proxies_list.append(proxy)
        else:
            continue
    return final_proxies_list
proxies_list = get_proxies(requests.get('https://free-proxy-list.net/'))

In [219]:
response = requests.get(page_link, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
link_list = []
obj_all = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['snippet-star', 'snippet-star--big'])
for obj in obj_all:
    way = 'https://simplewine.ru'+obj.attrs['href']
    way = way.replace('/reviews/','')
    link_list.append(way)

In [7]:
def getPageLinks(page_number):
    # составляем ссылку на страницу поиска
    link_list =[]
    page_link = 'https://simplewine.ru/catalog/vino/filter/color-krasnoe/page{}/'.format(page_number)
    response = requests.get(page_link, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
    soup = BeautifulSoup(response.content, 'html.parser')
    
    obj_all = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['snippet-star', 'snippet-star--big'])
    
    for obj in obj_all:
        way = 'https://simplewine.ru'+obj.attrs['href']
        way = way.replace('/reviews/','')
        link_list.append(way)
    return link_list

In [8]:
wine_list = []
for i in range(1,3):
    wine_list.extend(getPageLinks(i))
    time.sleep(1)

In [9]:
wine_list

['https://simplewine.ru/catalog/product/brunello_di_montalcino_2018_075_144520',
 'https://simplewine.ru/catalog/product/nipozzano_chianti_rufina_riserva_2020_075_143986',
 'https://simplewine.ru/catalog/product/brunello_di_montalcino_castelgiocondo_2018_075_143591',
 'https://simplewine.ru/catalog/product/sassi_chiusi_2015_075_145965',
 'https://simplewine.ru/catalog/product/marques_de_riscal_reserva_2019_075_144001',
 'https://simplewine.ru/catalog/product/le_volte_dell_ornellaia_2021_075_143635',
 'https://simplewine.ru/catalog/product/kaberne_sovinon_semeynyy_pezerv_2018_075_144776',
 'https://simplewine.ru/catalog/product/guidalberto_2021_075_144491',
 'https://simplewine.ru/catalog/product/two_hands_angel_s_share_2021_075_135214',
 'https://simplewine.ru/catalog/product/joseph_drouhin_bourgogne_pinot_noir_laforet_2021_075_140166',
 'https://simplewine.ru/catalog/product/domaine_clarence_dillon_clarendelle_by_haut_brion_rouge_2016_075_135650',
 'https://simplewine.ru/catalog/produ

In [34]:
sim = 'https://simplewine.ru/catalog/product/brunello_di_montalcino_2018_075_144520'
response = requests.get(sim, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
soup = BeautifulSoup(response.content, 'html.parser')

In [100]:
chars_1 = {'Стилистика вина':None,
 'Дегустационные характеристики':None,
 'Гастрономия':None,
 'Виноградники':None,
 'Способ выдержки':None,
 'Способ производства':None,
 'История этикетки':None,
 'Это интересно':None}

s_wine_1 = soup.findAll(attrs={'class':'characteristics-description__item'})

for obj in s_wine_1:

    name_char = obj.find(attrs={'class':'characteristics-description__item-title'})
    name_char = str(name_char).replace('<h4 class="characteristics-description__item-title">\n','').replace('</h4>','').replace(':','').strip()
    if name_char in chars_1:
        val = obj.find(attrs={'class':'characteristics-description__item-text'})
        val = str(val).replace('<div class="characteristics-description__item-text">\n','').replace('</div>','').replace(':','').strip()
        chars_1[name_char] = val

In [129]:
chars_1

{'Стилистика вина': 'красное — мощное, насыщенное, выдержанное в дубе',
 'Дегустационные характеристики': 'Вино интенсивного рубиново-красного цвета с легким пурпурным отблеском и комплексным ароматом, в котором раскрывается  целое богатство оттенков (красные ягоды, цветы, лакрица, специи), подчеркнутых легкими нотками сливы и ванили. Полнотелое, гармоничное, округлое во вкусе, с шелковистыми танинами, великолепной структурой и продолжительным послевкусием.',
 'Гастрономия': 'Можно порекомендовать к красному мясу на гриле, дичи и выдержанным сырам.',
 'Виноградники': 'Расположены на склонах холмов высотой 220 м в южной части области Монтальчино. Почвы мергелевые с большим количеством гальки. Кастелло Банфи - первое в мире винодельческое хозяйство, получившее международное признание за охрану окружающей среды, а также за этическую и социальную ответственность, что подтверждено соответствующим сертификатом (ISO 14001).',
 'Способ выдержки': 'В течение 24 месяцев в дубовых бочках различно

In [139]:
chars_2 = {'регион':None,
           'год':None,
           'крепость':None,
           'потенциал хранения':None,
           'аппелласьон':None,
           'категория':None,
           'декантация':None,
           'выдержка в емкости':None}

s_wine_2 = soup.find(attrs={'class':'characteristics-params__list'}).findAll(attrs={'characteristics-params__item'})

for obj in s_wine_2:
    name_char = re.search(r'\b[0-9a-zA-Zа-яёА-ЯЁ% ]*[:]',str(obj)).group().replace(':','').lower()
    if name_char in chars_2:
        val = obj.find('dd')
        val = re.search(r'\n*[>]\s*[0-9a-zA-Zа-яёА-ЯЁ% -.]+[<]',str(val)).group().replace('<','').replace('>','').replace('\n','').strip()
        chars_2[name_char] = val

In [141]:
chars_3 = soup.findAll(attrs={'class':'serving__suitable-title'})
temp = re.findall(r'[>]\n*[0-9a-zA-Zа-яёА-ЯЁ% ]*[<]',str(chars_3))
[i.replace('<','').replace('>','').replace('\n','').strip() for i in temp]

['Утка', 'Говядина', 'Выдержанный сыр']

In [ ]:
chars_4 = {'название':None,
           'цена':None,
           'рейтинг_SW':None,
           'год':None}

script = soup.find('script', {'data-skip-moving': '1', 'type': 'text/javascript'})
json_text = script.text.split('window.dataLayer = ', 1)[1].rsplit(';', 1)[0]
data = json.loads(json_text)
chars_4['цена'] = float(data[0]['ecommerce']['detail']['products'][0]['price'])
chars_4['название'] = data[0]['ecommerce']['detail']['products'][0]['name'].split(',')[0]
chars_4['год'] = float(data[0]['ecommerce']['detail']['products'][0]['variant'])
chars_4['рейтинг_SW'] =float(data[0]['ecommerce']['detail']['products'][0]['dimension13'])

In [ ]:
chars_4

In [ ]:
chars_5 = {'сладость':None,
           'кислотность':None,
           'танины':None,
           'тело':None}

temp = soup.findAll('li', attrs = {'class':'taste__item'})
for obj in temp:
    item = obj.find('div', class_='taste__title').text.strip().lower()
    if item in chars_5:
        chars_5[item] = len(re.findall(r'dot_active"',str(obj)))

In [ ]:
sim = 'https://simplewine.ru/catalog/product/brunello_di_montalcino_2018_075_144520'
#sim = 'https://simplewine.ru/catalog/product/chianti_classico_la_porta_di_vertine_2020_075_144075'
#sim = 'https://simplewine.ru/catalog/product/sedara_2021_075_143300'
response = requests.get(sim, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
soup = BeautifulSoup(response.content, 'html.parser')

In [245]:
sim += '/reviews/'

In [247]:
response = requests.get(sim, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
soup = BeautifulSoup(response.content, 'html.parser')

In [264]:
chars_6 = []
comments = soup.findAll('div', class_='product-review__item')
for comment in comments:
    res = comment.find('div', class_='product-review__item-comment').text.strip()
    chars_6.append(res)

In [265]:
chars_6

['Обязательно декантировать мин 60 минут. Иначе много кислоты. После декантирования вино играет совсем по другому.',
 'Очень органично, ярко и элегантно. В рамках цены соответствует на все 100%. Идеально подходит для романтичного вечера на двоих. Но перед употреблением дать подышать минимум  30 мин, что оно стало более ярким.',
 'Банфи - это достойное качество, их Брунелло отдельная песня, 5 баллов за эти деньги',
 'О да, это Брунелло)',
 'Шикарное вино. Аромат утреннего леса, дерева, тёмных ягод, кожи. Во вкусе ягоды, и чёрный чай в послевкусии.',
 'Отличное!',
 'Чудесное вино! Но ценник теперь сказочный))',
 'Самое лучшее',
 'Прекрасное вино Нового Света на каждый день, лёгкое, приятное.',
 'Отличный вкус']